## 3-1. 네이버 책 검색 API 호출하기 (필수)
###   검색어로  찾은  책 목록을 json 파일로 저장하기

In [ ]:
import urllib.request
import urllib.parse
import requests
import os
import json
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()


def search_books(query):
    
    payload={
        'query':query,
        'display':50,
        'sort':'sim'
    }
    
    
    
    client_id = os.getenv("NAVER_CLIENT_ID") # 개발자센터에서 발급받은 Client ID 값
    client_secret = os.getenv("NAVER_CLIENT_SECRET") # 개발자센터에서 발급받은 Client Secret 값
    
    headers ={
        'X-Naver-Client-Id' :client_id,
        'X-Naver-Client-Secret' : client_secret
    }
    
    url = "https://openapi.naver.com/v1/search/book.json" #?query={query}$display={display}"
    
    #request = urllib.request.Request(url)
    #response = urllib.request.urlopen(request)
    #rescode = response.getcode()
    
    res= requests.get(url, params=payload, headers=headers)
    
    if res.ok :
        items_data = res.json()['items']
        items_list = []
        for item in items_data :
            book_info = {
                'title': item['title'],
                'link': item['link'],
                'image': item['image'],
                'author': item['author'],
                'discount': item['discount'],
                'publisher': item['publisher'],
                'pubdate': item['pubdate'],
                'isbn': item['isbn'],
                'description': item['description']
            }
            items_list.append(book_info)
        
        save_dir = f'data/{query}'
        os.makedirs(save_dir,exist_ok=True)
        
        with open(f'{save_dir}/books.json','w',encoding='utf-8') as file:
            json.dump(items_list,file,ensure_ascii=False)
        
    else :
        print(f'Error Code : {res.status_code}')       
        
search_books('파이썬')

### books.json 파일을 Pandas DataFrame으로 저장하기

In [ ]:
import pandas as pd
book_df = pd.read_json('data/파이썬/books.json')

### 검색어로 찾은 책 목록 출력하기

In [ ]:
book_df.head()

###  검색어로  찾은  책 목록 중에서 가격이 2만원 이상인 책만 출력하기

In [ ]:
book_df.loc[book_df['discount'].values>=20000,['title','author','discount','publisher','pubdate']].sort_values(by='discount', ascending=False).reset_index(drop=True)

### 검색어로  찾은  책 목록 중에서 출판사가 인피니티북스인 책만 출력하기

In [ ]:
book_df.loc[book_df['publisher'] == '인피니티북스'].drop(columns=['image','description']).reset_index(drop=True)